# Bokeh graph data preparation

In [1]:
import networkx as nx
import json

import numpy as np

## Code to use in order to retrieve data from a particular debugging run
 
```{pyhton}
nx.write_gml('subgraph.gml')

import json

with open('subgraph.json', 'w') as fp:
    data = json.dump(readSequenceDb, fp)
```

Loading data from files generated during the debugging process. 

In [2]:
network = nx.read_gml('subgraph.gml')

with open('subgraph.json', 'r') as fp:
    readSequenceDb = json.load(fp)

Checking if the network nodes are in long verbose format or just a short one. In the former case, nodes will be renamed to shorter name containing only name of the sequences represented. 

In [3]:
flag = False

for node in network.nodes():
    if 'read_position' in node:
        flag = True
        break
        
        
if flag == True:
    mapping = {}
    for node in network.nodes():
        mapping[node] = node.split(';')[0]
    
    network = nx.relabel_nodes(network, mapping) 

## Retrieving the longest 'shortest path' through the graph

The following 

In [4]:
sp = dict(nx.all_pairs_shortest_path(network))

maxLength = 0
maxLengthNodes = []

for entry in sp:
           
    if maxLength < len(sp[entry]):
        maxLength = len(sp[entry])
        maxLengthNodes = []
        maxLengthNodes.append(list(sp[entry].keys()))
    elif maxLength == len(sp[entry]):
        maxLengthNodes.append(list(sp[entry].keys()))
                
    elif maxLength > len(sp[entry]):
        continue

In [5]:
spine = maxLengthNodes[0]

In [6]:
for node in spine:
    if len(list(network.predecessors(node))) == 0:
        tipNode1 = node
        break
        
for node in spine:
    if len(list(network.successors(node))) == 0:
        tipNode2 = node
        break       

In [7]:
shortest = list(nx.shortest_simple_paths(network,tipNode1, tipNode2))

In [8]:
endTips = []
beginningTips = []

for node in network.nodes():
    if len(list(network.successors(node))) == 0:
        endTips.append(node)
    elif len(list(network.predecessors(node))) == 0:
        beginningTips.append(node)

In [9]:
maxPathLen = 0
longestPath = []

for start in beginningTips:
    for end in endTips:
        try:
            paths = list(nx.shortest_simple_paths(network, start, end))
            
            pathLength = max(map(lambda x : len(x), paths))
                        
            if pathLength > maxPathLen:
                maxPathLen = pathLength
                                
                longestPath = []
                for entry in paths:
                    longestPath.append(entry)
            elif pathLength == maxPathLen:
                for entry in paths:
                    longestPath.append(entry)
                
        except:
            # No path between the two nodes has been found.
            continue

In [10]:
iterPath =iter(longestPath[0])

positionX = 0
positionY = 0

previousNode = longestPath[0][0]

nodePosition = {}

next(iterPath)



for node in iterPath:
    seqLen = len(readSequenceDb[previousNode])
    overhang = seqLen - network[previousNode][node]['overlap']
    positionX = positionX + overhang
    nodePosition[node] = (positionX, positionY)
    
    positionY = positionY + 10
    previousNode = node

In [11]:
nodeDict = {}

for key in nodePosition:

    nodeDict[key] = nodePosition[key]

In [12]:
nodesToIncorporate = list(network.nodes())

incorporatedNodes = []

for node in nodeDict:
    nodesToIncorporate.remove(node)
    incorporatedNodes.append(node)
    
print(incorporatedNodes)

['2167.1', '2922.1', '351.2', '233.2', '1298.1', '1479.2', '1446.1', '247.1', '2423.1', '2917.2', '351.1', '1735.1', '1648.1']


In [13]:
counter = 0

while len(nodesToIncorporate) > 0:
    
    tempDict = {}
    #print(nodePosition)
    
    for node in nodeDict:
        counter = counter + 1
        
        #print(len(nodesToIncorporate))
        
        for successor in network.successors(node):
            if successor not in incorporatedNodes:
                
                #print(successor)
                
                x, y = nodeDict[node]
                
                xNew = x + len(readSequenceDb[successor]) - network[node][successor]['overlap']
                yNew = y + 10
                
                tempDict[successor] = (xNew, yNew)
                nodesToIncorporate.remove(successor)
                incorporatedNodes.append(successor)
                
        for predecessor in network.predecessors(node):
            if predecessor not in incorporatedNodes:
                
                x, y = nodeDict[node]
                
                xNew = x - len(readSequenceDb[predecessor]) + network[predecessor][node]['overlap']
                yNew = y + 10 
                
                tempDict[predecessor] = (xNew, yNew)
                nodesToIncorporate.remove(predecessor)
                incorporatedNodes.append(predecessor)
    
    for entry in tempDict:
        #3print("Adding {}".format(entry))
        nodeDict[entry] = tempDict[entry]
        #print(len(nodeDict))
    
    if counter > 1000:
        print("TOO MANY ITERATIONS")
        break

In [14]:
layout = {}

for node in nodeDict:
    layout[node] = np.array([nodeDict[node][0], nodeDict[node][1]], dtype='int')

In [15]:
from bokeh.models import HoverTool,ColumnDataSource
from bokeh.plotting import show, figure

nodes, nodes_coordinates = zip(*sorted(layout.items()))
nodes_xs, nodes_ys = list(zip(*nodes_coordinates))

In [48]:
def get_nodes_specs(_network, _layout, _seqDict):
    d = dict(xs=[], ys=[], widths = [], name= [])
    
    nodes, nodes_coordinates = zip(*sorted(_layout.items()))
    d['xs'], d['ys'] = list(zip(*nodes_coordinates))
    
    for node in nodes:
        d['name'].append(node)
        d['widths'].append(len(_seqDict[node]))
    return d

def get_edges_specs(_network, _layout):
    d = dict(xs=[], ys=[], alphas=[], widths = [])
    weights = [d['overlap'] for u, v, d in _network.edges(data=True)]
    max_weight = max(weights)
    calc_alpha = lambda h: 1 + log(h/max_weight)/log(2)
    calc_width = lambda h: 1 + log(h/max_weight)/log(2)
    
    # example: { ..., ('user47', 'da_bjoerni', {'weight': 3}), ... }
    for u, v, data in _network.edges(data=True):
        d['xs'].append([_layout[u][0], _layout[v][0]])
        d['ys'].append([_layout[u][1], _layout[v][1]])
        d['alphas'].append(calc_alpha(data['overlap']))
        d['widths'].append(calc_width(data['overlap']))
    return d

In [ ]:

nodes_source = ColumnDataSource(dict(x=nodes_xs, y=nodes_ys,name=nodes))

In [50]:
nodes_source = ColumnDataSource(get_nodes_specs(_network=network, _layout=layout, _seqDict=readSequenceDb))

lines_source = ColumnDataSource(get_edges_specs(network, layout))


In [16]:
hover = HoverTool(tooltips=[('name', '@name')])
plot = figure(plot_width=1500, plot_height=500,
              tools=['tap', hover, 'box_zoom', 'reset'])

r_circles = plot.circle('xs', 'ys', source=nodes_source, size=width, color='blue', level = 'overlay')

r_lines = plot.quad()

r_lines = plot.multi_line('xs', 'ys', line_width='widths', alpha='alphas', color='gray',
                          source=lines_source)

In [18]:
from math import log



show(plot)

In [32]:
a = get_edges_specs(network, layout)

In [31]:
get_nodes_specs(_network=network, _layout=layout, _seqDict=readSequenceDb)

TypeError: unhashable type: 'dict'

In [21]:
from math import log

In [25]:
for u, v, data in network.edges(data=True):
    print('u: {} \n'.format(u))
    print('v: {} \n'.format(v))
    print('data: {} \n'.format(data))

u: 1298.1 

v: 1479.2 

data: {'overlap': 78} 

u: 1479.2 

v: 1446.1 

data: {'overlap': 89} 

u: 1735.1 

v: 1648.1 

data: {'overlap': 99} 

u: 417.2 

v: 1210.1 

data: {'overlap': 88} 

u: 1210.1 

v: 3057.1 

data: {'overlap': 79} 

u: 2917.2 

v: 351.1 

data: {'overlap': 89} 

u: 351.1 

v: 1735.1 

data: {'overlap': 82} 

u: 3079.1 

v: 886.1 

data: {'overlap': 84} 

u: 886.1 

v: 1990.1 

data: {'overlap': 91} 

u: 1446.1 

v: 247.1 

data: {'overlap': 99} 

u: 247.1 

v: 2423.1 

data: {'overlap': 88} 

u: 908.1 

v: 2801.2 

data: {'overlap': 86} 

u: 2801.2 

v: 647.1 

data: {'overlap': 93} 

u: 1758.2 

v: 2801.2 

data: {'overlap': 70} 

u: 1758.2 

v: 2167.1 

data: {'overlap': 76} 

u: 2499.2 

v: 2922.1 

data: {'overlap': 88} 

u: 2922.1 

v: 351.2 

data: {'overlap': 97} 

u: 2167.1 

v: 2922.1 

data: {'overlap': 81} 

u: 1990.1 

v: 3300.2 

data: {'overlap': 87} 

u: 351.2 

v: 233.2 

data: {'overlap': 99} 

u: 233.2 

v: 1298.1 

data: {'overlap': 93} 

u: 24

In [24]:
network.nodes()

NodeView(('1298.1', '1479.2', '1735.1', '1648.1', '417.2', '1210.1', '2917.2', '351.1', '3079.1', '886.1', '1446.1', '247.1', '908.1', '2801.2', '1758.2', '2499.2', '2922.1', '2167.1', '1990.1', '351.2', '233.2', '2423.1', '3300.2', '233.1', '1200.2', '1200.1', '2533.2|1751.2|1788.2', '647.1', '3057.1', '607.1', '283.2|1998.2|175.2|1998.2|283.2|175.2', '719.1', '840.2|3067.2|647.2'))